In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, LSTM, Reshape, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [14]:
IMG_SIZE = 224  # Adjust this based on your dataset
BATCH_SIZE = 32
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
EPOCHS = 200# Adjust number of epochs based on your dataset

In [15]:
IMG_SIZE = 128
BATCH_SIZE = 32
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
EPOCHS = 200


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image size and batch size
IMG_SIZE = (224, 224)  # You can change this to fit your model
BATCH_SIZE = 32


# Define paths to train and test directories (ensure the paths are correct)
# Update the paths to the absolute location
train_path = 'C:/Users/hp/Desktop/MTECH_ SEM 2/DNN/Diagnosis of Diabetic Retinopathy/train'
test_path = 'C:/Users/hp/Desktop/MTECH_ SEM 2/DNN/Diagnosis of Diabetic Retinopathy/test'


# Check if the directories exist (optional, for debugging)
if not os.path.exists(train_path):
    print(f"Train path does not exist: {train_path}")
else:
    print(f"Train path exists: {train_path}")

if not os.path.exists(test_path):
    print(f"Test path does not exist: {test_path}")
else:
    print(f"Test path exists: {test_path}")

# Initialize ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_data = datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,       # Resize images to IMG_SIZE
    batch_size=BATCH_SIZE,      # Define batch size
    class_mode='categorical'    # Multi-class classification
)

# Load testing data
test_data = datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,       # Resize images to IMG_SIZE
    batch_size=BATCH_SIZE,      # Define batch size
    class_mode='categorical',   # Multi-class classification
    shuffle=False               # Make sure test data isn't shuffled
)

# Number of classes (in your case it should be 2: DR and NO_DR)
NUM_CLASSES = train_data.num_classes
print(f"Number of classes: {NUM_CLASSES}")

# Optional: View class labels
print("Class labels:", train_data.class_indices)


Train path exists: C:/Users/hp/Desktop/MTECH_ SEM 2/DNN/Diagnosis of Diabetic Retinopathy/train
Test path exists: C:/Users/hp/Desktop/MTECH_ SEM 2/DNN/Diagnosis of Diabetic Retinopathy/test
Found 2076 images belonging to 2 classes.
Found 231 images belonging to 2 classes.
Number of classes: 2
Class labels: {'DR': 0, 'No_DR': 1}


In [25]:
# ------------------- MODEL BUILDING -------------------
def build_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

def build_dnn():
    model = Sequential([
        Flatten(input_shape=INPUT_SHAPE),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

def build_rnn():
    model = Sequential([
        Reshape((IMG_SIZE, IMG_SIZE * 3), input_shape=INPUT_SHAPE),
        LSTM(64, return_sequences=True),
        LSTM(64),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

def build_lenet():
    model = Sequential([
        Conv2D(6, (5, 5), activation='relu', input_shape=INPUT_SHAPE),
        MaxPooling2D(),
        Conv2D(16, (5, 5), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

def build_vgg16():
    base = VGG16(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    for layer in base.layers:
        layer.trainable = False
    x = base.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=preds)
    return model

def build_resnet():
    base = ResNet50(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    for layer in base.layers:
        layer.trainable = False
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    preds = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=preds)
    return model

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
                                     LSTM, Reshape, Input, BatchNormalization,
                                     GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.applications import VGG16, ResNet50

# ------------------- GLOBAL CONFIG -------------------
IMG_SIZE = 224
BATCH_SIZE = 32
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
EPOCHS = 200

# These should be defined from your dataset
# Example:
# train_data = ImageDataGenerator(...).flow_from_directory(...)
# test_data = ImageDataGenerator(...).flow_from_directory(...)
NUM_CLASSES = train_data.num_classes

# ------------------- MODEL BUILDING -------------------
def build_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

def build_dnn():
    model = Sequential([
        Flatten(input_shape=INPUT_SHAPE),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

def build_rnn():
    model = Sequential([
        Reshape((IMG_SIZE, IMG_SIZE * 3), input_shape=INPUT_SHAPE),
        LSTM(64, return_sequences=True),
        BatchNormalization(),
        LSTM(64),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

def build_lenet():
    model = Sequential([
        Conv2D(6, (5, 5), padding='same', activation='relu', input_shape=INPUT_SHAPE),
        MaxPooling2D(),
        Conv2D(16, (5, 5), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

def build_vgg16():
    base = VGG16(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    for layer in base.layers:
        layer.trainable = False
    x = Flatten()(base.output)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(NUM_CLASSES, activation='softmax')(x)
    return Model(inputs=base.input, outputs=preds)

def build_resnet():
    base = ResNet50(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
    for layer in base.layers:
        layer.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(NUM_CLASSES, activation='softmax')(x)
    return Model(inputs=base.input, outputs=preds)

# ------------------- TRAINING SETUP -------------------
models = {
    "CNN": build_cnn,
    "DNN": build_dnn,
    "RNN": build_rnn,
    "LeNet": build_lenet,
    "VGG16": build_vgg16,
    "ResNet": build_resnet
}

optimizers = {
    "adam": Adam,
    "sgd": SGD,
    "rmsprop": RMSprop
}

learning_rates = [0.001, 0.01]
results = []
history_logs = {}
model_summaries = {}
speed_logs = {}

# ------------------- CONFUSION MATRIX PLOT -------------------
def plot_confusion_matrix(y_true, y_pred, classes, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title(f'Confusion Matrix - {title}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()
    plt.show()

# ------------------- TRAIN & EVALUATE -------------------
def train_and_evaluate(model_fn, optimizer_fn, lr, model_name, label):
    model = model_fn()
    model_summaries[f"{model_name}_{label}"] = [layer.__class__.__name__ for layer in model.layers]

    if optimizer_fn:
        optimizer = optimizer_fn(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    else:
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

    start_time = time.time()
    history = model.fit(train_data, epochs=EPOCHS, validation_data=test_data, verbose=0)
    duration = time.time() - start_time

    loss, acc = model.evaluate(test_data, verbose=0)
    history_logs[f"{model_name}_{label}"] = history
    speed_logs[f"{model_name}_{label}"] = duration

    preds = model.predict(test_data)
    y_pred = np.argmax(preds, axis=1)
    y_true = test_data.classes
    class_labels = list(test_data.class_indices.keys())

    plot_confusion_matrix(y_true, y_pred, class_labels, f"{model_name}_{label}")
    print(f"Classification Report - {model_name}_{label}:")
    print(classification_report(y_true, y_pred, target_names=class_labels))

    return loss, acc, model

# ------------------- MAIN TRAINING LOOP -------------------
trained_models = {}

for model_name, model_fn in models.items():
    print(f"===== Training {model_name} WITHOUT Optimizer =====")
    try:
        loss, acc, model = train_and_evaluate(model_fn, None, None, model_name, "none")
        results.append((model_name, "None", "None", loss, acc))
        trained_models[f"{model_name}_none"] = model
        print(f"✅ {model_name} no optimizer - Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    except Exception as e:
        print(f"⚠️ Error in {model_name} without optimizer: {e}")

    for opt_name, opt_class in optimizers.items():
        for lr in learning_rates:
            print(f"===== Training {model_name} with {opt_name.upper()} (lr={lr}) =====")
            try:
                loss, acc, model = train_and_evaluate(model_fn, opt_class, lr, model_name, f"{opt_name}_{lr}")
                results.append((model_name, opt_name, lr, loss, acc))
                trained_models[f"{model_name}_{opt_name}_{lr}"] = model
                print(f"✅ {model_name} + {opt_name} - Loss: {loss:.4f}, Accuracy: {acc:.4f}")
            except Exception as e:
                print(f"⚠️ Error in {model_name} with {opt_name} (lr={lr}): {e}")


===== Training CNN WITHOUT Optimizer =====


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Plot individual Accuracy and Loss graphs per model
for key, history in history_logs.items():
    plt.figure(figsize=(10, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Val')
    plt.title(f'Accuracy - {key}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Val')
    plt.title(f'Loss - {key}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Merged Accuracy & Loss Graphs
plt.figure(figsize=(12, 6))

# Merged Accuracy
for key, history in history_logs.items():
    plt.plot(history.history['val_accuracy'], label=f'{key}')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Merged Loss
plt.figure(figsize=(12, 6))
for key, history in history_logs.items():
    plt.plot(history.history['val_loss'], label=f'{key}')
plt.title('Validation Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Create summary report table
summary_df = pd.DataFrame(results, columns=['Model', 'Optimizer', 'Learning Rate', 'Loss', 'Accuracy'])
summary_df = summary_df.sort_values(by='Accuracy', ascending=False)
summary_df.reset_index(drop=True, inplace=True)

print("\n📋 Model Performance Summary:")
display(summary_df)

# Optional: Save to CSV
# summary_df.to_csv('model_summary_report.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# -------------------------------
# 1. Plot Accuracy & Loss per Model
# -------------------------------
for key, history in history_logs.items():
    plt.figure(figsize=(10, 4))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Val')
    plt.title(f'Accuracy - {key}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Val')
    plt.title(f'Loss - {key}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# -------------------------------
# 2. Merged Accuracy Graph (only)
# -------------------------------
plt.figure(figsize=(14, 7))
for key, history in history_logs.items():
    plt.plot(history.history['val_accuracy'], label=key)
plt.title('📈 Validation Accuracy Comparison Across Models')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right', fontsize='small')
plt.grid(True)
plt.tight_layout()
plt.show()

# -------------------------------
# 3. Summary Table
# -------------------------------
summary_df = pd.DataFrame(results, columns=['Model', 'Optimizer', 'Learning Rate', 'Loss', 'Accuracy'])
summary_df = summary_df.sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

print("\n📋 Model Performance Summary:")
display(summary_df)

# Optional: Save report
# summary_df.to_csv("model_accuracy_summary.csv", index=False)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# -------------------------------
# 1. Load a sample image
# -------------------------------
img_path = r"C:\Users\hp\Desktop\MTECH_ SEM 2\DNN\Diagnosis of Diabetic Retinopathy\test\DR\362c4a96cebb_png.rf.192f091fc13737d8c432a2b9c16bd035.jpg"

img = image.load_img(img_path, target_size=(224, 224))  # Resize to match model input

# Show image
plt.imshow(img)
plt.axis('off')
plt.title("Sample Input Image")
plt.show()

# -------------------------------
# 2. Preprocess the image
# -------------------------------
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize if needed

# -------------------------------
# 3. Predict using a trained model
# -------------------------------
model_name = 'CNN_adam_0.001'  # use any available trained model
model = trained_models[model_name]

pred = model.predict(img_array)
predicted_class = np.argmax(pred, axis=1)[0]

# -------------------------------
# 4. Display Prediction
# -------------------------------
class_labels = list(test_data.class_indices.keys())  # example: ['dr', 'no_dr']
predicted_label = class_labels[predicted_class]

print(f"✅ Predicted Label: {predicted_label}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# -------------------------------
# 1. Load the No_DR image
# -------------------------------
img_path = r"C:\Users\hp\Desktop\MTECH_ SEM 2\DNN\Diagnosis of Diabetic Retinopathy\valid\No_DR\4d167ca69ea8_png.rf.ba34a6446a9941030c1b8601d0211fb1.jpg"

img = image.load_img(img_path, target_size=(224, 224))  # Resize to match model input

# Show image
plt.imshow(img)
plt.axis('off')
plt.title("Sample Input Image (No_DR)")
plt.show()

# -------------------------------
# 2. Preprocess the image
# -------------------------------
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

# -------------------------------
# 3. Loop through all models to predict
# -------------------------------
class_labels = list(test_data.class_indices.keys())  # ['dr', 'no_dr']

# Predict with all models
for model_name, model in trained_models.items():
    pred = model.predict(img_array)
    predicted_class = np.argmax(pred, axis=1)[0]
    predicted_label = class_labels[predicted_class]
    print(f"Model: {model_name} - Predicted Label: {predicted_label}")
